In [ ]:
import vk_api
import random
import io
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api import VkUpload
import asyncio

In [21]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import os
from PIL import Image
import io
import hashlib
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from msedge.selenium_tools import Edge, EdgeOptions

edge = webdriver.Edge(EdgeChromiumDriverManager().install())

# All in same directory

def fetch_image_urls(i, j:int, query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    i = i+1
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)        
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = list()
    image_count = 0
    results_start = 0
    error_clicks = 0
    while (image_count < max_links_to_fetch) & (error_clicks < 1): # error clicks to stop when there are no more results to show by Google Images. You can tune the number
        scroll_to_end(wd)
        if i == j:
            break
        print('Starting search for Images')

        # get all image thumbnail results
        thumbnail_results = wd.find_elements(By.CSS_SELECTOR, "img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        for img in thumbnail_results[results_start:max_links_to_fetch]:
            # try to click every thumbnail such that we can get the real image behind it
            print("Total Errors till now:", error_clicks)
            try:
                print('Trying to Click the Image')
                img.click()
                time.sleep(sleep_between_interactions)
                print('Image Click Successful!')
            except Exception:
                error_clicks = error_clicks + 1
                print('ERROR: Unable to Click the Image')
                if(results_start < number_results):
                	continue
                else:
                	break
                	
            results_start = results_start + 1

            # extract image urls    
            
            print('Extracting of Image URLs')
            actual_images = wd.find_elements(By.CSS_SELECTOR, 'img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                    image_urls.append(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            print('Current Total Image Count:', image_count)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
            else:
                load_more_button = wd.find_elements(By.CSS_SELECTOR, ".mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")
            	        
        results_start = len(thumbnail_results)
    j = j + 1
    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
import GoogleImageScraper

In [22]:
def upload_photo(upload, url):
    img = requests.get(url).content
    f = io.BytesIO(img)

    response = upload.photo_messages(f)[0]

    owner_id = response['owner_id']
    photo_id = response['id']
    access_key = response['access_key']

    return owner_id, photo_id, access_key


def send_photo(vk, peer_id, owner_id, photo_id, access_key):
    attachment = f'photo{owner_id}_{photo_id}_{access_key}'
    vk.method('messages.send', {'peer_id' : peer_id, 'attachment' : attachment, 'random_id' : 0})

In [ ]:
# API-ключ созданный ранее
TOKEN = "vk1.a.nZaPVrXoWSqCaT2k5mS3GkCTdGTJRFJSSQKHujTBYCThvCUeU4z1XXyMbs0iJJfS1j2LvKNvO4kUipA4gkDmssgOd38EJZqS2RaBuD2M1S1S9UkAeRUbIOEgrcEXwpR28etoUCqvHVy1YM1u6cWsQYO2LZW3kYluwztAGVst7MfZOGqUZyEejRjeuA-HO1ghJXdZulEwkYrg1xHi7jlOJA"

# Авторизуемся как сообщество
vk = vk_api.VkApi(token=TOKEN)
mcounter1 = 1
mcounter2 = 0

# Работа с сообщениями
longpoll = VkLongPoll(vk)

for event in longpoll.listen():
    if event.type == VkEventType.MESSAGE_NEW: 
        zapros = event.text
        upload = VkUpload(vk)
        usid = event.user_id 
        if mcounter1 < mcounter2:
            send_photo(vk, usid, *upload_photo(upload, fetch_image_urls(mcounter1, mcounter2, zapros, 1, edge)[0]))
            
     
